# DT Assignment1

# Data Loading

In [1]:
import pandas as pd 
import numpy as np

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


# 1. Gini 계수를 구하는 함수 만들기

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [3]:
def get_gini(df, label):
    label = df[label].value_counts() # class label 분포
    
    gini = 1 # gini index 공식에 의해
    for value in label.values: # label의 각 class의 개수
        gini -= (value / label.values.sum()) ** 2 # gini index 공식
    
    return gini    

In [4]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

# 2. Feature의 Class를 이진 분류로 만들기
 ## ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [5]:
from itertools import combinations 

def get_binary_split(df, attribute):
    items = set(df[attribute]) # attribute의 모든 class 집합
    result = [x for length in range(1, len(items)) 
            for x in combinations(items, length)]
    # 길이가 1부터 class 개수-1인 모든 부분집합을 itertools library를 통해 생성
    return result

In [6]:
get_binary_split(pd_data,"age")

[('senior',),
 ('middle_aged',),
 ('youth',),
 ('senior', 'middle_aged'),
 ('senior', 'youth'),
 ('middle_aged', 'youth')]

# 3. 다음은 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [7]:
def get_attribute_gini_index(df, attribute, label):
    result = {} # 결과를 담을 dict
    binary_split = get_binary_split(df, attribute) # 이진 분류 결과
    gini1 = gini2 = None # 이진 분류의 첫번째와 두번째 gini 계수
    for i in range(len(binary_split)):
        b = binary_split[i] # 이진 분류 결과의 원소
        name = '_'.join(b) # dict의 key를 이진 분류 원소들을 _로 연결한 것으로 설정
        d1 = pd.DataFrame() # 첫번째 이진 분류 class만을 담는 데이터프레임
        for j in b:
            d = df[df[attribute] ==  j] # attribute이 j인 데이터만 뽑음
            d1 = pd.concat([d1, d])  # b에 속하는 class만 추가
            
        d2 = df.drop(d1.index) # d1을 제외한 나머지 데이터
        gini1 = get_gini(d1, label) # 이진 분류의 첫 번째 gini 계수 구하기
        gini2 = get_gini(d2, label) # 이진 분류의 두 번째 gini 계수 구하기
        
        result[name] = (len(d1) / len(df)) * gini1 + (len(d2) / len(df)) * gini2 
    # 최종 gini 계수 구해서 dictionary 형태로 result에 저장
    return result

In [8]:
get_attribute_gini_index(pd_data, "age", "class_buys_computer")

{'senior': 0.4571428571428572,
 'middle_aged': 0.35714285714285715,
 'youth': 0.3936507936507937,
 'senior_middle_aged': 0.3936507936507937,
 'senior_youth': 0.35714285714285715,
 'middle_aged_youth': 0.4571428571428572}

여기서 가장 작은 Gini index값을 가지는 class를 기준으로 split해야겠죠?

In [9]:
min(get_attribute_gini_index(pd_data, "age", "class_buys_computer").items())

('middle_aged', 0.35714285714285715)

# 다음의 문제를 위에서 작성한 함수를 통해 구한 값으로 보여주세요!
## 문제1) 변수 ‘income’의 이진분류 결과를 보여주세요.

## 문제2) 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.

## 문제3) 문제 2에서 제시한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각   다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요.

In [10]:
## 문제1 답안
get_binary_split(pd_data, "income")

[('high',),
 ('low',),
 ('medium',),
 ('high', 'low'),
 ('high', 'medium'),
 ('low', 'medium')]

In [11]:
## 문제2 답안
ginis = {} # 각 attribute 별 gini 계수를 담을 dict
for attribute in pd_data.columns[:-1]: # label을 제외한 attribute들
    ginis[attribute] = min(get_attribute_gini_index(pd_data, attribute, pd_data.columns[-1]).values())
    # attribute를 key로, 그에 해당하는 가장 작은 gini 계수 값을 value로 dict에 추가
print(min(zip(ginis.values(), ginis.keys())))

(0.35714285714285715, 'age')


In [12]:
split_class = min(get_attribute_gini_index(pd_data, min(ginis), pd_data.columns[-1]))
split_class

'middle_aged'

In [13]:
## 문제3 답안
data1 = pd_data[pd_data[min(ginis)] == split_class]
data2 = pd_data.drop(data1.index)
# 문제 2에서 제시한 feature로 DataFrame을 split

ginis1 = {}
for attribute in pd_data.columns[:-1]:
    if attribute != min(ginis): 
    # data1의 경우 min(ginis), 즉 'age'의 class가 'middle_aged'밖에 없으므로
        ginis1[attribute] = min(get_attribute_gini_index(data1, attribute, data1.columns[-1]).values())
print(ginis1)
# 모든 class가 yes이므로 더이상 분류할 필요가 없다

ginis2 = {}
for attribute in pd_data.columns[:-1]:
    ginis2[attribute] = min(get_attribute_gini_index(data2, attribute, data2.columns[-1]).values())
print(ginis2)

{'income': 0.0, 'student': 0.0, 'credit_rating': 0.0}
{'age': 0.48, 'income': 0.375, 'student': 0.31999999999999984, 'credit_rating': 0.4166666666666667}


In [14]:
print(min(zip(ginis1.values(), ginis1.keys())))
print(min(zip(ginis2.values(), ginis2.keys())))
# 두 번째 split의 경우 student가 다음으로 가장 중요한 변수이다.

(0.0, 'credit_rating')
(0.31999999999999984, 'student')
